In [47]:
import pandas as pd
import pybit
from datetime import datetime

/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [8]:
from pybit.unified_trading import HTTP
session = HTTP(testnet=False)
resp = session.get_tickers(
    category="spot",
#    symbol="BTCUSDT",
)


In [17]:
tickers = []
for symbol in resp['result']['list']:
    ticker = symbol['symbol']
    if 'USDT' in ticker:
        volume = float(symbol['volume24h'])
        if volume > 1e06 :
            #print(ticker)
            tickers.append(ticker)
            

In [33]:
from pybit.unified_trading import HTTP
session = HTTP(testnet=True)
print(session.get_kline(
    category="spot",
    symbol="BTCUSDT",
    interval=60,
    start=1670601600000,
    end=1670608800000,
))

{'retCode': 0, 'retMsg': 'OK', 'result': {'category': 'spot', 'symbol': 'BTCUSDT', 'list': [['1670608800000', '16886', '16886', '16596.12', '16783.5', '323.156445', '5456578.71657288'], ['1670605200000', '16976.88', '17044.13', '15400', '16886', '323.936083', '5458892.50591019'], ['1670601600000', '17000', '17042.55', '16832.19', '16976.88', '1343.886676', '22846667.14058788']]}, 'retExtInfo': {}, 'time': 1712130663816}


In [41]:
from datetime import datetime, timedelta
def get_times():
    # Получение timestamp и ms на текущее время
    now = datetime.now()
    timestamp_now = int(now.timestamp()) * 1000
    ms_now = now.microsecond
    
    # Получение timestamp на неделю раньше
    week_ago = now - timedelta(days=7)
    timestamp_week_ago = int(week_ago.timestamp()) * 1000
    return ( timestamp_week_ago, timestamp_now)


In [87]:
st, et = get_times()
datetime.fromtimestamp(st/1000)
print("Start:", datetime.fromtimestamp(st/1000))
print("End:", datetime.fromtimestamp(et/1000))

Start: 2024-03-27 08:19:19
End: 2024-04-03 08:19:19


In [43]:
session = HTTP(testnet=True)
resp = session.get_kline(
    category="spot",
    symbol="BTCUSDT",
    interval= 720,
    start = st,
    end = et,
)


In [55]:
session = HTTP()
prices = {}
for ticker in tickers:
    resp = session.get_kline(
    category="spot",
    symbol= ticker,
    interval= 720,
    start = st,
    end = et,
    )

    df = pd.DataFrame (resp['result']['list'], columns = ['date', 'open', 'high','low','close','volume', 'turnover'])
    prices[ticker] = df.close

df_res = pd.DataFrame(prices)

In [63]:
df_res = df_res.astype(float)
df_ret = df_res.pct_change(fill_method = None)

In [75]:
dfx = df_ret[-2:]>0
df_up = df_ret[-2:][dfx].dropna(axis = 1)

In [77]:
df_up.sum()

MELOSUSDT     0.092576
SYNRUSDT      0.100216
TIAUSDT       0.017804
RATSUSDT      0.041698
FMCUSDT       0.049116
TUSDT         0.113071
LOOKSUSDT     0.094702
SCAUSDT       0.049543
MNTUSDT       0.073866
ERTHAUSDT     0.053692
CTCUSDT       0.015254
ANKRUSDT      0.083059
THNUSDT       0.571551
WOOUSDT       0.047579
FLTUSDT       0.207235
JTOUSDT       0.040013
TOKENUSDT     0.217989
MPLXUSDT      0.216104
DOMEUSDT      0.040649
PUMLXUSDT     0.048246
TONUSDT       0.008202
MANTAUSDT     0.040139
GLMRUSDT      0.049330
SOSUSDT       0.079796
MVLUSDT       0.019710
GMRXUSDT      0.166276
WENUSDT       0.159000
ACHUSDT       0.046232
SOLUSDT       0.016986
JASMYUSDT     0.030029
HVHUSDT       0.030520
ALTUSDT       0.047846
MBSUSDT       0.246159
APRSUSDT      0.141805
AGLAUSDT      0.031641
ICPUSDT       0.117399
LDOUSDT       0.056069
TURBOSUSDT    0.043483
PYTHUSDT      0.030486
RPKUSDT       0.001377
WAXPUSDT      0.034840
IRLUSDT       0.049717
ARKMUSDT      0.072684
THETAUSDT  

In [81]:
df_res['synrusdt'.upper()]

0     0.000608
1     0.000601
2     0.000565
3     0.000579
4     0.000604
5     0.000662
6     0.000675
7     0.000654
8     0.000660
9     0.000619
10    0.000655
11    0.000706
12    0.000749
13    0.000798
14    0.000826
Name: SYNRUSDT, dtype: float64